In [ ]:
# --- Predecir sobre las imágenes capturadas ---
results_list = []
image_paths = sorted([str(p) for p in Path(carpeta).glob('*.jpg')])
print('Imágenes encontradas:', image_paths)
for img_path in image_paths:
    print('Procesando:', img_path)
    try:
        if backend == 'ultralytics':
            res = model(img_path)[0]  # objeto Results de ultralytics
            boxes = getattr(res, 'boxes', None)
            if boxes is None or len(boxes) == 0:
                results_list.append({'image': img_path, 'class': None, 'conf': 0.0})
                continue
            confidences = boxes.conf.tolist()
            classes = boxes.cls.tolist()
            top_idx = int(np.argmax(confidences))
            top_conf = float(confidences[top_idx])
            top_cls = int(classes[top_idx])
            names = getattr(model, 'names', {}) if hasattr(model, 'names') else {}
            top_name = names.get(top_cls, str(top_cls))
        elif backend in {'torch_module','torch_state_dict'}:
            model.eval()
            img = cv2.imread(img_path)
            if img is None:
                raise RuntimeError('No se pudo leer la imagen para inferencia.')
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            tensor = torch.from_numpy(img_rgb).float().permute(2,0,1).unsqueeze(0) / 255.0
            with torch.no_grad():
                raw = model(tensor)
            # Suponemos salida estilo YOLO (bs, anchors, (5 + num_cls))
            if isinstance(raw, (list, tuple)) and len(raw) > 0:
                raw = raw[0]
            if raw.dim() == 4:
                raw = raw[0]
            raw = raw.cpu().numpy().reshape(-1, raw.shape[-1])
            boxes = raw[:, :4]
            obj_conf = raw[:, 4]
            class_scores = raw[:, 5:]
            scores = obj_conf[:, None] * class_scores
            top_idx = int(np.argmax(scores))
            top_conf = float(np.max(scores))
            top_cls = int(np.argmax(class_scores[top_idx]))
            names = getattr(model, 'names', {}) if hasattr(model, 'names') else {}
            top_name = names.get(top_cls, str(top_cls))
        else:
            raise RuntimeError('Backend no soportado para predicción.')
        results_list.append({'image': img_path, 'class': top_name, 'conf': top_conf})
        print(f' -> {top_name} ({top_conf:.3f})')
    except Exception as e:
        print('Error procesando', img_path, e)
        results_list.append({'image': img_path, 'class': None, 'conf': 0.0})

print('Predicción completada sobre todas las imágenes.')